<h1 style="color:#0084bb">Regression in all Shapes and Sizes</h1>

This notebook contains the first assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with some of the simplest neural networks possible. 
Essentially, these simple networks come down to a well-known tool in statistics: **regression**.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

%matplotlib auto

In [ ]:
# data generation


def plane_data(n=100, d=2, bound=3., noise=.1, offset=0., seed=None):
    """
    Generate samples for a linear regression dataset.
    
    Parameters
    ----------
    n : int, optional
        Number of samples to generate.
    d : int, optional
        Number of dimensions in the input data.
    bound : float, optional
        Distance between center value and extreme values in the inputs.
    noise : float, optional
        Standard deviation for the additive Gaussian noise.
    offset : float, optional
        Amount by which to shift the output values.
    seed : int, optional
        Set this number to get reproducable results.
        
    Returns
    -------
    x : (n, d) ndarray
        Input data.
    y : (n, 1) ndarray
        Target values.
    """
    rng = np.random.RandomState(seed or np.random.randint(256) + 1)
    x = rng.uniform(-bound, bound, size=(n, d))
    y = offset + x.mean(axis=-1) / bound
    target = y + noise * rng.randn(n) + offset
    return x, target.reshape(-1, 1)


def blob_data(n_per_blob=50, d=2, k=2, noise=.1, offset=0., seed=None):
    """
    Generate samples for a classification dataset.
    
    Parameters
    ----------
    n_per_blob : int, optional
        Number of samples per class to generate.
    d : int, optional
        Number of dimensions in the input data.
    k : int, optional
        Number of classes in the data.
    noise : float, optional
        Standard deviation for the additive Gaussian noise.
    offset : float, optional
        Amount by which to shift the input values.
    seed : int, optional
        Set this number to get reproducable results.
        
    Returns
    -------
    x : (n, d) ndarray
        Input data.
    y : (n, ) ndarray
        Target Labels.
    """
    means = np.zeros(shape=(k, d)) + np.linspace(-1, 1, k)[:, None]
    means += offset * d
    stds = 10 * d / k * noise + np.zeros_like(means)
    
    rng = np.random.RandomState(seed or np.random.randint(256) + 1)
    z = rng.randn(n_per_blob, k, d)
    x = stds * z + means
    y = np.ones(z.shape[:-1], dtype='int') * np.arange(k)
    return x.reshape(-1, d), y.flatten()

In [ ]:
# visual functions


def show_1d_model(x, y, w, b, pred_func):
    """Visualise 1D model on data"""
    plt.scatter(x, y, color='steelblue')
    _x = np.linspace(x.min(), x.max())[:, None]
    _s = my_first_network(_x, w, b)
    _pred = pred_func(_s)
    plt.plot(_x, _pred, color='tomato', linewidth=3, label='prediction')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.legend()

    
def show_2d_model(x, y, w, b, pred_func, cmap='viridis'):
    """Visualise 2D model on data"""
    _x = np.linspace(x.min(), x.max())
    _x1, _x2 = np.meshgrid(_x, _x)
    _x = np.c_[_x1.flat, _x2.flat]
    _s = my_first_network(_x, w, b)
    _pred = pred_func(_s)
    _pred = _pred.reshape(_x1.shape)
    vmin = min(_pred.min(), y.min())
    vmax = max(_pred.max(), y.max())
    plt.contour(_x1, _x2, _pred, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.scatter(*x.T, c=y.flatten(), cmap=cmap, vmin=vmin, vmax=vmax, edgecolors='w')
    plt.colorbar().set_label('$y$')
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.legend(handles=[plt.Line2D([], [], color='black', label='prediction contours')])

<h2 style="color:#0084bb">Linear Regression</h2>

Linear regression is a natural choice for modelling linear relationships in statistics. Ater all, it maximises the likelihood of a linear model if the noise in the data is assumed to be Gaussian. On top of that, the *Maximum Likelihood Estimator* can be written down analytically. Since this model corresponds to a single-layer neural network without activation function, it is a good starting point for exploring neural networks.

<h6 style="color:#0084bb">Model</h6>

Concretely, the outputs $\boldsymbol{y} \in \mathbb{R}^K$ are assumed to depend on the inputs $\boldsymbol{x} \in \mathbb{R}^D$ and random noise $\varepsilon_k \sim \mathcal{N}(0, \sigma^2)$ so that

$$\boldsymbol{y} = f(\boldsymbol{x}) + \boldsymbol{\varepsilon},$$

where $f : \mathbb{R}^D \to \mathbb{R}^K$ is some linear function. 
To model the underlying relationship $f$, we will use one of the simplest possible *Neural Networks*:

$$g(\boldsymbol{x} \mathbin{;} \theta) = \boldsymbol{w} \cdot \boldsymbol{x} + \boldsymbol{b},$$

where $\theta = \{\boldsymbol{w}, \boldsymbol{b}\}$ is the set of parameters for the model.
**Note** that this network corresponds to a linear regression model.

<h6 style="color:#0084bb">Likelihood</h6>

The *Likelihood* of a supervised machine learning model for a dataset with 
*identically and independenltly distributed* inputs $\boldsymbol{X}$ and outputs $\boldsymbol{Y}$ is given by

$$\mathcal{L}(\theta \mathbin{;} \boldsymbol{X}, \boldsymbol{Y}) = \prod_{n = 1}^{N} p(\boldsymbol{x}^n, \boldsymbol{y}^n \mathbin{;} \theta) = \prod_{n = 1}^{N} p(\boldsymbol{y}^n \mid \boldsymbol{x}^n \mathbin{;} \theta).$$

This likelihood quantises how likely some data is, given the parameters of a model.
Given the model assumptions as stated above, the conditional probablity 
in the likelihood product can be written as 

$$p(\boldsymbol{y}^n \mid \boldsymbol{x}^n \mathbin{;} \theta) = p_\mathcal{N}\left(\boldsymbol{y}^n \mathbin{;} g(\boldsymbol{x}^n \mathbin{;} \theta), \sigma^2\right) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(\frac{-\left(\boldsymbol{y}^n - g(\boldsymbol{x}^n \mathbin{;} \theta)\right)^2}{2 \sigma^2}\right).$$

<h6 style="color:#0084bb">Optimisation</h6>

Maximising the likelihood, $\mathcal{L}(\theta)$, is equivalent to maximising the *log-likelihood*, $\mathcal{l}(\theta) = \ln \mathcal{L}(\theta)$, or minimising the additive inverse. Therefore, the optimal parameters, $\theta^*$ are specified by

$$\begin{align}
  \theta^* & = \arg\min_\theta \left\{-\mathcal{l}(\theta \mathbin{;} \boldsymbol{X}, \boldsymbol{Y}) \right\} \\
  & = \arg\min_\theta \left\{-\sum_{n = 1}^{N} \ln p(\boldsymbol{y}^n \mid \boldsymbol{x}^n \mathbin{;} \theta) \right\} \\
  & = \arg\min_\theta \left\{N \ln \sqrt{2 \pi \sigma^2} + \frac{1}{2 \sigma^2} \sum_{n = 1}^{N}\left(\boldsymbol{y}^n - g(\boldsymbol{x}^n \mathbin{;} \theta)\right)^2 \right\},
\end{align}$$

which is equivalent to minimising the sum of squared errors

$$\theta^* = \arg\min_\theta \left\{\frac{1}{2} \sum_{n = 1}^{N}\left(\boldsymbol{y}^n - g(\boldsymbol{x}^n \mathbin{;} \theta)\right)^2 \right\}.$$

<h3 style="color:#0084bb">Exercise 1: Numpy Refreshment (2 Points)</h3>

The model described above should be straightforward to implement. How about you get familiar with [python](https://docs.python.org/3) and [numpy](https://docs.scipy.org/doc/numpy) (again)?

> Implement the simple neural network from above as well as the squared error loss.

In [ ]:
def squared_error(prediction, truth):
    """
    Compute the squared error of a predicted value, given the actual target.
    
    Parameters
    ----------
    prediction : (N, K) ndarrary
        The value(s) predicted by the model.
    truth : (N, K) ndarray
        The actual target(s) from the data.
        
    Returns
    -------
    error : (N, K) ndarray
        The squared error(s) for the prediction(s).
    """
    #[[(t11 - p11)**2, (t12 - p12)**2],
    # [(t21 - p21)**2, (t22 - p22)**2]]
    error = np.subtract(truth, prediction)
    error = np.square(error)
    return error


def my_first_network(x, w, b):
    """
    Predict a value for some input with a simple neural network.
    
    Parameters
    ----------
    x : (N, D) ndarray
        The input samples to predict a value for.
    w : (K, D) ndarray
        The input parameters for the neural network.
    b : (K, ) ndarray
        The bias parameter for the neural network.
        
    Returns
    -------
    prediction : (N, K) ndarray
        The predicted value for the input from the network.
    """
    # w × x + b or x × w**T + b
    prediction = x.dot(w.T)
    prediction = np.add(prediction, b)
    return prediction

In [ ]:
x, y = plane_data(d=1, seed=1856)
w, b = np.random.randn(y.shape[1], x.shape[1]), np.random.randn(y.shape[1])

# print mse and visualise model
pred = my_first_network(x, w, b)
print(np.mean(squared_error(pred, y), axis=0).item())
show_1d_model(x, y, w, b, lambda x: x)

<h3 style="color:#0084bb">Exercise 2: Analytical Solution (3 Points)</h3>

Minimising the negative log-likelihood comes down to solving linear regression with the *least-squares* method, i.e. by minimising the sum of squared residuals. For the least-squares method in linear regression, the optimal parameters can be derived analytically. The optimal solution is given by:

$$\theta^* = (\boldsymbol{X}^T \boldsymbol{X})^{-1} \boldsymbol{X}^T \boldsymbol{Y}.$$

Does this model incorporate a bias parameter? Where can the bias parameter be found?

> Implement a function that computes this analytical solution for given inputs and outputs. The function should allow solutions both with and without bias parameter.

**Hint:** to get help for commands, execute a cell with following code (for help on `np.linalg.inv`):
```python
?np.linalg.inv
```

In [ ]:
def analytical_solution(x, y, bias=True):
    """
    Get the optimal parameters for linear regression,
    given input data and target values.
    
    Parameters
    ----------
    x : (N, D) ndarray
        Input data to the network.
    y : (N, K) ndarray
        Target values.
    bias : bool, optional
        Whether or not the network should have a bias term.
        
    Returns
    -------
    w : (K, D) ndarray
        The optimal input parameters for the network.
    b : (K, ) ndarray
        The optimal bias parameters for the network
        or zero if `bias=False`.
    """
    dim_N = x.shape[0]
    dim_D = x.shape[1]
    dim_K = y.shape[1]
    assert dim_N == y.shape[0], \
        "N dimension of input x and y differs: x" + \
        str(x.shape) + " y" + str(y.shape)
    #theta = ((x.T.dot(x))**-1).dot(x.T).dot(y) 
    theta = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)# D×K
    if bias:
        ############
        #b = np.dot(x, theta)# N×D.D×K
        #b = np.subtract(y, b)
        #print("dim before np.mean: " + str(b.shape))
        #b = np.mean(a=b, axis=0, keepdims=True)
        #print("dim after np.mean:  " + str(b.shape))
        ############
        # b = b.T; cause it should be 1×K (row vector) (as it stands in moodle)
        # but cause this example is with K := 1, I can't say.
        b = np.mean(a=np.subtract(y, np.dot(x, theta)), axis=0, keepdims=True).T
    else:
        b = np.array([[0] * dim_K]) # right shape 1×K
    w = theta.T
    
    assert w.shape[0] == dim_K and w.shape[1] == dim_D, \
        "Missmatch of dimensions for W" + str(w.shape) + \
        " should be (" + str(dim_K) + " ," + str(dim_D) + ")"
    assert b.shape[1] == dim_K and b.shape[0] == 1, \
        "Missmatch of dimension for B" + str(b.shape) + \
        " should be (" + str(dim_K) + ", 1)"
    return w, b

In [ ]:
x, y = plane_data(d=1, seed=1856)
w, b = analytical_solution(x, y, bias=True)
pred = my_first_network(x, w, b)
print(np.mean(squared_error(pred, y), axis=0).item())
show_1d_model(x, y, w, b, lambda x: x)

<h3 style="color:#0084bb">Exercise 3: Importance of Bias (1 Point)</h3>

Whether `bias=True` or `bias=False` in the fragment above, there seems to be little difference. Why is it important to have a bias parameter? 

> Configure the arguments for generating data below to illustrate the problem of not having a bias parameter.

**Note:** The plots below are [contour plots](https://en.wikipedia.org/wiki/Contour_line). The colours encode the value of the function in each point.

In [ ]:
# Bias corresponds to Offset, that is y = offset + ...
# Without bias, we can't get an output unequal zero for an input of zero.
# set n to 1000 to see the difference better (more dots)
# set offset to a negative value like -1
# we can see on the left nearly matching colors for the dots and the lines,
# on the right the do not match by color.
x, y = plane_data(n=1000, bound=3., noise=.1, offset=-1., seed=1856)

In [ ]:
plt.figure(figsize=(14, 5))  # you can change figure dimensions if necessary
# visualise model with bias
w, b = analytical_solution(x, y, bias=True)
pred = my_first_network(x, w, b)
mse = np.mean(squared_error(pred, y), axis=0)
plt.subplot(121)
plt.title(f"mse: {mse.item():.5f}")
show_2d_model(x, y, w, b, lambda x: x)
# visualise model without bias
w, b = analytical_solution(x, y, bias=False)
pred = my_first_network(x, w, b)
mse = np.mean(squared_error(pred, y), axis=0)
plt.subplot(122)
plt.title(f"mse: {mse.item():.5f}")
show_2d_model(x, y, w, b, lambda x: x)

<h2 style="color:#0084bb">Logistic Regression</h2>

The values to predict are not always continuous. A very common task in machine learning is to answer yes-no questions, e.g. Is this an image of a cat? These problems are known as *binary classification* tasks. This can be encoded by labelling positive examples with a 1 and negative examples with a 0. As a result, the target values are Bernoulli distributed, rather than Gaussian. This also means that the relationship is no longer linear and linear regression makes little sense. This is where *Logistic Regression* comes in play.

<h6 style="color:#0084bb">Model</h6>

Mathematically, we assume that the relation between inputs and outputs can be modelled as follows:

$$\boldsymbol{y} = \begin{cases}
  1 & f(\boldsymbol{x}) > 0.5 \\
  0 & f(\boldsymbol{x}) \leq 0.5
\end{cases}.$$

Since the relation is non-linear, we add an additional non-linearity, $\sigma : \mathbb{R} \to \mathbb{R}$, to our simple network so that

$$g(\boldsymbol{x} \mathbin{;} \theta) = \sigma\left(\boldsymbol{w} \cdot \boldsymbol{x} + \boldsymbol{b}\right),$$

where the parameters are again $\theta = \{\boldsymbol{w}, \boldsymbol{b}\}$. The non-linear function maps real values to values between zero and one. This allows the outputs of the network to be interpreted as the probability for its input to be classified as positive. For logistic regression, this non-linearity is the logistic sigmoid, which is defined as 

$$\sigma(x) = \frac{1}{1 + e^{-x}}.$$

In [ ]:
def logistic_sigmoid(s):
    """
    Compute the logistic sigmoid function.
    
    Parameters
    ----------
    s : ndarray
        The logits to apply the logistic sigmoid function on.
    
    Returns
    -------
    a : ndarray
        The probabilitie(s) for the given logit(s).
    """
    return 1 / (1 + np.exp(-s))

<h3 style="color:#0084bb">Exercise 4: Logistic Likelihood Maximisation (3 Points)</h3>


Just as for linear regression, maximum likelihood estimation can be used to find parameters for a model that solves the binary classification problem. Is there an error function for logistic regression, like we have the squared error for linear regression? Time to try out some [$\LaTeX$](https://www.overleaf.com/learn)!

> Derive the optimisation objective for maximum likelihood in a similar way the squared error was derived for linear regression. Implement the resulting *logistic error* in a python function.


To derive the maximum likelihood estimator, we need the conditional probability of the label:
> For my understanding, we use y (that has only values of {0, 1}) as k. Because we have for the function we need to implement (and I first implemented it and thought on the math behind on a little note will doing it) only the values of y and g(x; theta) are given, such that for me only this makes sense - but I think also, I'm terrible wrong.

$$\begin{align}
  p(\boldsymbol{y}^n \mid \boldsymbol{x}^n \mathbin{;} \theta) & = g(\boldsymbol{x^n} \mathbin{;} \theta)^{y^n} \cdot (1 - g(\boldsymbol{x^n} \mathbin{;} \theta))^{1 - y^n}
\\
  & = \sigma\left(\boldsymbol{w} \cdot \boldsymbol{x^n} + \boldsymbol{b}\right)^{y^n} \cdot (1 - \sigma\left(\boldsymbol{w} \cdot \boldsymbol{x^n} + \boldsymbol{b}\right))^{1-y^n}
\end{align}$$

The negative log-likelihood is then given by:
$$\begin{align}
  -\mathcal{l}(\theta \mathbin{;} \boldsymbol{X}, \boldsymbol{Y})
  & = -\sum_{n = 1}^{N} \ln p(\boldsymbol{y}^n \mid \boldsymbol{x}^n \mathbin{;} \theta) \\
  & = -\sum_{n = 1}^{N} \ln \sigma\left(\boldsymbol{w} \cdot \boldsymbol{x^n} + \boldsymbol{b}\right)^{y^n} \cdot (1 - \sigma\left(\boldsymbol{w} \cdot \boldsymbol{x^n} + \boldsymbol{b}\right))^{1-y^n}
\end{align}$$

**Hint**: the probability mass function of the Bernoulli distribution, $p_\mathrm{B}(k \mathbin{;} p) = \begin{cases} 1 - p & k = 0 \\ p & k = 1 \end{cases}$ can be written as $p_\mathrm{B}(k \mathbin{;} p) = p^k (1 - p)^{1 - k}$

<h5 style="color:#0084bb">Some Notes on LaTeX in Jupyter Notebooks</h5>

$\LaTeX$ is an advanced typesetting system that can be used for all sorts of documents. One of the key features of $\LaTeX$ is the ability to insert mathematical formulas in the text. Jupyter notebooks allow to display mathematical symbols (and some other things) through $\LaTeX$ syntax. Technically the conversion from $\LaTeX$ code to HTML is done by [MathJax](https://www.mathjax.org/), so if you cannot see the nice formulas I put in this notebook, you might need to troubleshoot MathJax.

To give you an idea of the possibilities of $\LaTeX$:

| code                   | rendered             | code (big)               | rendered (big)         |
|:---------------------- | --------------------:|:------------------------ | ----------------------:|
| `$\LaTeX$`             | $\LaTeX$             |                          |                        |
| `$\ln \boldsymbol{x}$` | $\ln \boldsymbol{x}$ | `$$\ln \boldsymbol{x}$$` | $$\ln \boldsymbol{x}$$ |
| `$\frac{a}{b}$`        | $\frac{a}{b}$        | `$$\frac{a}{b}$$`        | $$\frac{a}{b}$$        |
| `$\sum_{i=1}^I i^2$`   | $\sum_{i=1}^I i^2$   | `$$\sum_{i=1}^I i^2$$`   | $$\sum_{i=1}^I i^2$$   |

If you have troubles to find the right symbols, you can use [detexify](http://detexify.kirelabs.org/classify.html) or directly use a more graphical $\LaTeX$ equation editor, e.g. [this one from codecogs](https://www.codecogs.com/latex/eqneditor.php)

In [ ]:
def logistic_error(prediction, truth):
    """
    Compute the logistic error of a predicted value, given the actual target.
    
    Parameters
    ----------
    prediction : (N, K) ndarrary
        The value(s) predicted by the model.
    truth : (N, K) ndarray
        The actual target(s) from the data.
        
    Returns
    -------
    error : (N, K) ndarray
        The logistic error(s) for the prediction(s).
    """
    # truth is y: are values of {0, 1}
    # prediction is allready sigmoided
    #
    # if truth: prediction
    # else:     1 - prediction
    dim_N = prediction.shape[0]
    dim_K = prediction.shape[1]

    assert truth.shape[0] == dim_N, str(truth.shape[0]) + " != " + str(dim_N)
    assert truth.shape[1] == dim_K, str(truth.shape[1]) + " != " + str(dim_K)
    
    t = np.power(prediction, truth)
    f = np.power(np.subtract(1, prediction), np.subtract(1, truth))
    error = np.multiply(t, f)
    
    assert error.shape[0] == dim_N, str(error.shape[0]) + " != " + str(dim_N)
    assert error.shape[1] == dim_K, str(error.shape[1]) + " != " + str(dim_K)
    return error

In [ ]:
# fixed version as posted in moodle
x, y = blob_data(d=1, seed=1856)
y = y.reshape(-1,1)
w, b = np.random.randn(1, x.shape[1]), np.random.randn(1)

# print mle and visualise model
logits = my_first_network(x, w, b)
pred = logistic_sigmoid(logits)
print(np.mean(logistic_error(pred, y), axis=0))
show_1d_model(x, y, w, b, logistic_sigmoid)

<h3 style="color:#0084bb">Exercise 5: Gradient Descent (3 Points)</h3>

Since there is no closed form solution for logistic regression, other methods are necessary to find the optimal parameters. Gradient descent is one of the simplest methods to get to a good solution. By pushing the parameters in the opposite direction of the gradient, it is possible to find local minima relatively fast.

Concretely, the gradient descent algorithm computes the gradients for some inputs and moves a certain amount in the direction of that gradient. This process is repeated a number of times until convergence. The amount of gradient to go down is known as the *learning rate*, and a single step of computing gradients and updating parameters is commonly called an *epoch*.

> Implement the gradient descent step, i.e. one epoch, for logistic regression. Make sure not to forget any parameters in the gradient descent!

In [ ]:
def gradient_descent(step_func, x, y, bias=True, epochs=10, lr=1.):
    """
    Use gradient descent to compute the weights of a network.
    
    Parameters
    ----------
    step_func : callable
        Function that implements the gradient descent step.
        Signature should correspond to `err = step_func(x, y, w, b, lr)`.
    x : ndarray
        Inputs to the network.
    y : ndarray
        Targets for the network.
    bias : bool, optional
        Whether or not the network should have a bias term.
    epochs : int, optional
        Number of steps to descent.
    lr : float, optional
        Scaling factor for gradient steps.
        
    Returns
    -------
    w : ndarray
        The optimal input parameters for the network.
    b : ndarray, optional
        The optimal bias parameters for the network.
        Only returned if `bias=True`.
    """
    w = np.zeros((y.shape[1], x.shape[1]))
    b = np.zeros(y.shape[1])
    
    for i in range(epochs):
        err = step_func(x, y, w, b, lr=lr)
        
        if not bias:
            # undo bias updates
            b[:] = 0
            
        # print mean error at most 10 times during learning
        if epochs < 10 or i % (epochs // 10) == 0:
            print(np.mean(err, axis=0).item())
        
    return w, b


def logistic_gradient_descent_step(x, y, w, b, lr=1.):
    """
    Compute the gradient of the logistic error w.r.t. the parameters
    and apply update to the parameters.
    
    Parameters
    ----------
    x : (N, D) ndarray
        Inputs to the network.
    y : (N, K) ndarray
        Targets for the network.
    w : (K, D) ndarray
        Input weights of the network.
    b : (K, ) ndarray
        Bias weights of the network.
    lr : float, optional
        Scaling factor for gradient steps.
        
    Returns
    -------
    err : (N, K) ndarray
        The error of the prediction from the network 
        for the given input data before the update.
        
    Notes
    -----
    Updates the network parameters in-place.
    """
    logits = my_first_network(x, w, b) # N×K
    pred = logistic_sigmoid(logits)    # N×K
    err = logistic_error(pred, y)      # N×K
    
    ##
    # w
    dim_N = x.shape[0]
    grad = (np.dot(x.T, np.subtract(y, pred)) ) * lr # D×K
    w += grad.T # K×D, why plus not minus? but the result looks fine
    
    # b - nothing found in our scripts, but this seams to work, multiplicate with lr is to small
    b[:] += np.mean(a=np.subtract(y, pred), axis=0, keepdims=False) #* lr # 1×K
    ##
    
    return err


def logistic_gradient_descent(x, y, bias=True, epochs=10, lr=1e-3):
    return gradient_descent(logistic_gradient_descent_step, x, y[:, None], bias, epochs, lr)

In [ ]:
x, y = blob_data(d=1, seed=1856)
w, b = logistic_gradient_descent(x, y, epochs=100, bias=True)
show_1d_model(x, y, w, b, logistic_sigmoid)

<h2 style="color:#0084bb">Softmax Regression</h2>

Softmax regression is essentially a generalisation of binary logistic regression that allows to solve *multi-class problems*, i.e. questions with more than two possible (exclusive) answers. Instead of predicting a single probability, the goal in softmax regression is to predict a probability for each of the possible classes. 

<h3 style="color:#0084bb">Exercise 6: One-hot Encoding (1 Point)</h3>

Since the labels are mostly integer values, some encoding is necessary to have the targets reflect probabilities. E.g. if there are three classes, the labels would be `0`, `1`, `2`. The most common way to encode these labels as probability vectors is to use a one-hot encoding. A one-hot code uses as much bits as there are labels. The code is zero everywhere, except at the index that corresponds to the label, where it is one. E.g. in the case of the three zero-indexed labels, the encoding for the label `1` would be `[0, 1, 0]`

> Implement a function to compute one-hot encodings from integer labels.

In [ ]:
def to_one_hot(y, k=None):
    """
    Compute a one-hot encoding from a vector of integer labels.
    
    Parameters
    ----------
    y : (N, ) ndarray
        The zero-indexed integer labels to encode.
    k : int, optional
        The number of distinct labels in `y`.
        
    Returns
    -------
    one_hot : (N, k) ndarray
        The one-hot encoding of the labels.
    """
    n = len(y)
    if k is None:
        k = len(np.unique(y))
    
    #mapping = [-1] * k
    #for i in range(n):
    #    if y[i] in mapping:
    #        pos = mapping.index(y[i])
    #    else:
    #        pos = mapping.index(-1)
    #        mapping[pos] = y[i]
    #    row = [0] * k
    #    row[pos] = 1
    #    if i == 0:
    #        one_hot = [row]
    #    else:
    #        one_hot = np.vstack([one_hot, row])
    for i in range(n):
        row = [0] * k
        row[y[i]] = 1
        if i == 0:
            one_hot = [row]
        else:
            one_hot = np.vstack([one_hot, row])
    
    return one_hot

In [ ]:
y = np.random.randint(3, size=5)
print("original y:")
print(y)
print("result:")
print(to_one_hot(y, 3))

<h3 style="color:#0084bb">Exercise 7: Softmax Gradients (2 Bonus Points)</h3>

Time to generalise logistic gradient descent to handle multiple labels! Either you remember these functions from the lecture or you can freshen up your memory in the script. 

> Implement the functions necessary for training a softmax classifier with gradient descent. Make sure not to forget any parameters in the gradient descent!

In [ ]:
def softmax(s):
    """
    Compute the softmax function.
    
    Parameters
    ----------
    s : (N, K) ndarray
        The logits to apply the softmax function on.
    
    Returns
    -------
    a : (N, K) ndarray
        The probabilitie(s) for the given logit(s).
    """
    dim_N = s.shape[0]
    dim_K = s.shape[1]
    s = np.exp(s)
    div = np.sum(s, axis=1, keepdims=True)
    assert div.shape[0] == dim_N
    assert div.shape[1] == 1
    s = np.divide(s, div)
    
    np.testing.assert_allclose(np.sum(s, axis=1), 1)
    assert s.shape[0] == dim_N
    assert s.shape[1] == dim_K
    return s


def cross_entropy(prediction, truth):
    """
    Compute the logistic error of a predicted value, given the actual target.
    
    Parameters
    ----------
    prediction : (N, K) ndarrary
        The value(s) predicted by the model.
    truth : (N, K) ndarray
        The actual target(s) from the data.
        
    Returns
    -------
    error : (N, ) ndarray
        The logistic error(s) for the prediction(s).
    """
    # Hint: check the sign!
    raise NotImplementedError("TODO: implement cross_entropy function!")


def softmax_gradient_descent_step(x, y, w, b, lr=1.):
    """
    Compute the gradient of the cross-entropy w.r.t. the parameters
    and apply update to the parameters.
    
    Parameters
    ----------
    x : (N, D) ndarray
        Inputs to the network.
    y : (N, K) ndarray
        Targets for the network.
    w : (K, D) ndarray
        Input weights of the network.
    b : (K, ) ndarray
        Bias weights of the network.
    lr : float, optional
        Scaling factor for gradient steps.
        
    Returns
    -------
    err : (N, ) ndarray
        The error of the prediction from the network 
        for the given input data before the update.
        
    Notes
    -----
    Updates the network parameters in-place.
    """
    logits = my_first_network(x, w, b)
    pred = softmax(logits)
    err = cross_entropy(pred, y)
    
    raise NotImplementedError("TODO: implement softmax_gradient_descent_step function!")
    
    return err

def softmax_gradient_descent(x, y, bias=True, epochs=10, lr=1e-3):
    return gradient_descent(softmax_gradient_descent_step, x, y, bias, epochs, lr)

In [ ]:
num_classes = 2  # feel free to play around!
x, y = blob_data(k=num_classes, seed=1856)
y_hot = to_one_hot(y, k=num_classes)
w, b = softmax_gradient_descent(x, y_hot, epochs=100, bias=True)
show_2d_model(x, y, w, b, lambda x: np.sum(softmax(x) * np.arange(num_classes), axis=-1))